# Beforeware

> FastHTML beforeware for API key management

In [ ]:
#| default_exp middleware.beforeware

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, List, Callable, Any
from functools import wraps

from cjm_fasthtml_byok.core.storage import BYOKManager
from cjm_fasthtml_byok.core.security import validate_environment
from cjm_fasthtml_byok.core.types import KeyNotFoundError

## Beforeware Functions

In [ ]:
#| export
def create_byok_beforeware(
    byok_manager: BYOKManager  # The BYOK manager instance
):
    """Create a FastHTML beforeware handler for BYOK functionality."""
    def handler(
        req,  # FastHTML request object
        sess  # Session object
    ):
        """
        FastHTML beforeware function.
        Adds byok manager to request scope.
        """
        req.scope['byok'] = byok_manager
        # Don't return anything! FastHTML expects None for success
    
    return handler

## Route Decorators

In [ ]:
#| export
def require_api_key(
    provider: str,  # The provider name to check for
    user_id_func: Optional[Callable] = None  # Optional function to get user_id from request Usage: @rt @require_api_key("openai") def chat(request): byok = request.scope['byok'] api_key = byok.get_key(request, "openai") # Use the API key...
):
    """Decorator that requires an API key to be present for a route."""
    def decorator(
        func: Callable  # The route handler function to decorate
    ):
        """Decorator function that wraps the route handler"""
        @wraps(func)
        def wrapper(
            request: Any,  # FastHTML request object
            *args,
            **kwargs
        ):
            """Wrapper that checks for API key before calling the route handler"""
            if 'byok' not in request.scope:
                raise RuntimeError("BYOK beforeware not configured")
            
            byok = request.scope['byok']
            user_id = user_id_func(request) if user_id_func else None
            
            if not byok.has_key(request, provider, user_id):
                from fasthtml.common import Titled, Card, P, A
                return Titled(
                    "API Key Required",
                    Card(
                        P(f"Please provide your {provider} API key to continue."),
                        A("Configure API Keys", href="/settings/api-keys")
                    )
                )
            
            return func(request, *args, **kwargs)
        return wrapper
    return decorator

In [ ]:
#| export
def require_any_api_key(
    providers: List[str],  # List of provider names to check
    user_id_func: Optional[Callable] = None  # Optional function to get user_id from request Usage: @rt @require_any_api_key(["openai", "anthropic", "google"]) def chat(request): # Use whichever API key is available pass
):
    """Decorator that requires at least one of the specified API keys."""
    def decorator(
        func: Callable  # The route handler function to decorate
    ):
        """Decorator function that wraps the route handler"""
        @wraps(func)
        def wrapper(
            request: Any,  # FastHTML request object
            *args,
            **kwargs
        ):
            """Wrapper that checks for any API key before calling the route handler"""
            if 'byok' not in request.scope:
                raise RuntimeError("BYOK beforeware not configured")
            
            byok = request.scope['byok']
            user_id = user_id_func(request) if user_id_func else None
            
            # Check if any of the providers has a key
            has_any = any(
                byok.has_key(request, provider, user_id)
                for provider in providers
            )
            
            if not has_any:
                from fasthtml.common import Titled, Card, P, Ul, Li, A
                return Titled(
                    "API Key Required",
                    Card(
                        P("Please provide at least one of the following API keys:"),
                        Ul(*[Li(provider) for provider in providers]),
                        A("Configure API Keys", href="/settings/api-keys")
                    )
                )
            
            return func(request, *args, **kwargs)
        return wrapper
    return decorator

## Security Beforeware

In [ ]:
#| export
class SecurityCheckBeforeware:
    """
    Beforeware that performs security checks.
    """
    
    def __init__(
        self,
        require_https: bool = True,  # Whether to require HTTPS in production
        is_production: Optional[bool] = None  # Whether running in production (auto-detected if None)
    ):
        """Initialize security check beforeware with HTTPS requirements"""
        self.require_https = require_https
        self.is_production = is_production
    
    def __call__(
        self,
        request: Any,  # FastHTML request object
        session: Any  # Session object
    ):
        """
        Validate security environment on each request.
        """
        try:
            validate_environment(
                request,
                require_https=self.require_https,
                is_production=self.is_production
            )
        except Exception:
            # Log but don't block requests
            pass
        
        return request, session

## Cleanup Beforeware

In [ ]:
#| export
class CleanupBeforeware:
    """
    Beforeware that cleans up expired keys.
    """
    
    def __init__(
        self,
        byok_manager: BYOKManager,  # The BYOK manager instance
        user_id_func: Optional[Callable] = None  # Optional function to get user_id from request
    ):
        """Initialize cleanup beforeware with BYOK manager"""
        self.byok = byok_manager
        self.user_id_func = user_id_func
    
    def __call__(
        self,
        request: Any,  # FastHTML request object
        session: Any  # Session object
    ):
        """
        Clean up expired keys on each request.
        """
        if self.byok.config.auto_cleanup:
            try:
                user_id = self.user_id_func(request) if self.user_id_func else None
                providers = self.byok.list_providers(request, user_id)
                
                # Check each provider and let storage handle expiration
                for provider in providers:
                    self.byok.get_key(request, provider, user_id)
            except Exception:
                # Don't block requests due to cleanup errors
                pass
        
        return request, session

## Complete Setup Helper

In [ ]:
#| export
def setup_byok(
    secret_key: str,  # Application secret key
    db: Optional[Any] = None,  # Optional database for persistent storage
    user_id_func: Optional[Callable] = None,  # Optional function to get user_id from request
    enable_security_checks: bool = True,  # Enable HTTPS checking
    enable_cleanup: bool = True  # Enable automatic cleanup of expired keys
):
    """Complete setup helper for BYOK with FastHTML. Returns beforeware functions and the BYOK manager."""
    from cjm_fasthtml_byok.core.types import BYOKConfig
    
    # Create manager
    byok = BYOKManager(secret_key, db)
    
    # Create list of beforeware handlers
    beforeware_list = []
    
    # Add main BYOK beforeware
    beforeware_list.append(create_byok_beforeware(byok))
    
    if enable_security_checks:
        security_handler = SecurityCheckBeforeware()
        beforeware_list.append(security_handler)
    
    if enable_cleanup:
        cleanup_handler = CleanupBeforeware(byok, user_id_func)
        beforeware_list.append(cleanup_handler)
    
    return beforeware_list, byok

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()